# 1. 설정, 유틸 함수들

In [2]:
#%pip install numpy pandas matplotlib seaborn scikit-learn mglearn pyarrow tensorflow keras tqdm

In [ ]:
# ===== 셀 1: 설정 + 유틸 =====
import os, json, math
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import pandas as pd

In [ ]:


# ▼ CSV 파일이 있는 폴더와 파일명(원하면 수정)
ROOT = "."
FILES = [("ygs.csv", "YGS"), ("kum.csv", "KUM"), ("hws.csv", "HWS"), ("icn.csv", "ICN")]

# ▼ 출력 폴더
OUTDIR = Path("preprocessed")
OUTDIR.mkdir(parents=True, exist_ok=True)

# --- CSV 읽기(자동 구분자 감지 + 컬럼명 정규화: 소문자/특수문자→_) ---
def sniff_read_csv(path: str) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        print(f"[INFO] 파일 없음: {path}")
        return None
    try:
        df = pd.read_csv(path, sep=None, engine="python")
        df.columns = (
            df.columns
            .str.strip()
            .str.lower()
            .str.replace(r"[^0-9a-zA-Z_]+", "_", regex=True)
        )
        return df
    except Exception as e:
        print(f"[ERROR] 읽기 실패 {path}: {e}")
        return None

# --- 데이터프레임 요약 출력 ---
def print_df_summary(name: str, df: pd.DataFrame, head_rows: int = 3) -> None:
    print(f"\n=== {name} | shape={df.shape} ===")
    print("Columns:", list(df.columns))
    print("Sample rows:")
    print(df.head(head_rows).to_string(index=False))

# --- 필수 컬럼 표준화(정확한 이름으로 강제) ---
#   주신 실제 컬럼 목록에 맞춰 '그대로' 매핑합니다.
#   - epc_code        ← 'epc_code'
#   - event_time      ← 'event_time'
#   - event_type      ← 'event_type'
#   - location_id     ← 'location_id'  (이름 유지; category로 사용)
REQUIRED = ["epc_code", "event_time", "event_type", "location_id"]

def standardize_required(df: pd.DataFrame) -> pd.DataFrame:
    # 이미 소문자/언더스코어 처리된 상태라는 가정하에, 필수 컬럼 확인
    missing = [c for c in REQUIRED if c not in df.columns]
    if missing:
        raise KeyError(f"필수 컬럼이 없습니다: {missing}\n"
                    f"현재 컬럼: {list(df.columns)}")
    # 불필요한 공백/형 변환 등은 아래 단계에서 처리
    return df

# --- 시간 파싱 + 시계열 피처(Δt, 시간대 sin/cos) ---
def ensure_dt(series: pd.Series) -> pd.Series:
    return pd.to_datetime(series, errors="coerce", utc=False)

def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    if not {"epc_code", "event_time"}.issubset(df.columns):
        raise KeyError("시간 피처 생성을 위해 'epc_code'와 'event_time'이 필요합니다.")
    out = df.copy()
    out["event_time"] = ensure_dt(out["event_time"])
    # EPC별 시간 정렬
    out.sort_values(["epc_code", "event_time"], inplace=True, kind="stable")
    # Δt(초) — 시퀀스 시작은 0
    out["delta_t_sec"] = out.groupby("epc_code")["event_time"].diff().dt.total_seconds().fillna(0.0)
    # 시간대(0~23) → sin/cos
    hours = out["event_time"].dt.hour.astype(float)
    out["hour_sin"] = (2 * math.pi * hours / 24.0).apply(math.sin)
    out["hour_cos"] = (2 * math.pi * hours / 24.0).apply(math.cos)
    return out

# --- 간단 어휘집 생성기 ---
def build_vocab(series: pd.Series, name: str, min_freq: int = 1) -> Dict:
    """
    - 예약 토큰: <PAD>=0, <UNK>=1
    - 정렬: 빈도 내림차순 → (동률 시) 토큰 문자열 오름차순
    - location_id는 숫자여도 '코드'이므로 문자열로 변환 후 카테고리로 취급합니다.
    """
    s = series.astype(str)
    counts = s.value_counts(dropna=False)
    counts = counts[counts >= min_freq]
    items = sorted(counts.items(), key=lambda kv: (-int(kv[1]), str(kv[0])))
    id2token = ["<PAD>", "<UNK>"] + [str(k) for k, _ in items]
    return {
        "name": name,
        "reserved": ["<PAD>", "<UNK>"],
        "id2token": id2token,
        "counts": {str(k): int(v) for k, v in counts.items()},
    }

# --- 어휘집을 이용해 ID 인코딩 ---
def encode_with_vocab(series: pd.Series, vocab: Dict, colname: str) -> pd.Series:
    token2id = {tok: i for i, tok in enumerate(vocab["id2token"])}
    unk_id = vocab["reserved"].index("<UNK>") if "<UNK>" in vocab["reserved"] else 1
    return series.astype(str).map(lambda x: token2id.get(x, unk_id)).astype("int32")

# --- 저장 도우미(Parquet 우선, 실패 시 CSV 폴백) ---
def save_table(df: pd.DataFrame, path_parquet: Path, path_csv: Optional[Path] = None):
    try:
        df.to_parquet(path_parquet, index=False)
        print(f"[OK] 저장(parquet): {path_parquet} (shape={df.shape})")
    except Exception as e:
        if path_csv is None:
            path_csv = path_parquet.with_suffix(".csv")
        df.to_csv(path_csv, index=False)
        print(f"[WARN] parquet 저장 실패({e}) → CSV로 저장: {path_csv} (shape={df.shape})")


# 2. CSV 읽기 & 요약 / 병합 → 필수 컬럼 표준화 → 저장

In [ ]:
# ===== 셀 2: CSV 읽기 & 요약 =====
import pandas as pd
print("[INFO] 모든 CSV 파일을 읽어 하나의 테이블로 병합합니다...")

dfs = []
for fname, factory in FILES:
    path = os.path.join(ROOT, fname)
    df = sniff_read_csv(path)
    if df is None:
        continue
    print_df_summary(f"{fname} ({factory})", df)
    # factory가 데이터에 별도 컬럼으로 있을 수도 있지만, 파일별 라벨을 임시로 붙여둡니다.
    df["__factory_label__"] = factory
    dfs.append(df)

if not dfs:
    raise SystemExit("[STOP] CSV를 하나도 찾지 못했습니다. 파일 경로/이름을 확인하세요.")

# ===== 병합/표준화/저장 =====
merged = pd.concat(dfs, ignore_index=True)
print(f"\n[INFO] 병합 shape: {merged.shape}")

# 필수 컬럼 확인/고정(정확한 이름 사용)
std = standardize_required(merged)

# factory 최종화: 원본에 'factory'가 없다면, 임시 라벨 사용
if "factory" not in std.columns:
    std["factory"] = std.pop("__factory_label__")
else:
    std.drop(columns=["__factory_label__"], errors="ignore", inplace=True)

# [수정 제안] Parquet 저장을 위해 데이터 타입을 명시적으로 변환
# 문제가 될 만한 object 타입 컬럼을 문자열(str)로 바꿔줍니다.
for col in std.select_dtypes(include=['object']).columns:
    print(f"[DEBUG] Converting column '{col}' to string type for Parquet compatibility.")
    std[col] = std[col].astype(str)

# 저장
save_table(std, OUTDIR / "combined.parquet")
print("[OK] 데이터 병합 및 기본 표준화 완료.")



=== ygs.csv (YGS) | shape=(132200, 18) ===
Columns: ['_scan_location', 'location_id', 'hub_type', 'business_step', 'event_type', 'operator_id', 'device_id', 'epc_code', 'epc_header', 'epc_company', 'epc_product', 'epc_lot', 'epc_manufacture', 'epc_serial', 'product_name', 'event_time', 'manufacture_date', 'expiry_date']
Sample rows:
_scan_location  location_id    hub_type business_step  event_type  operator_id  device_id                                      epc_code  epc_header  epc_company  epc_product  epc_lot  epc_manufacture  epc_serial product_name          event_time    manufacture_date  expiry_date
          양산공장            3 YGS_Factory       Factory Aggregation            3          3 001.8805843.2932031.100001.20250701.000000001           1      8805843      2932031   100001         20250701           1    Product 1 2025-07-01 10:23:39 2025-07-01 10:23:39     20251231
          양산공장            3 YGS_Factory       Factory Aggregation            3          3 001.8809437.120319

# 3. 시간 피처(Δt, hour sin/cos) 생성 → 저장 / 어휘집 생성(JSON) + ID 인코딩 컬럼 추가 → 저장

In [ ]:
import pandas as pd
import json

# ===== 셀 3: 시간 피처 생성/저장 =====

print("\n[INFO] 시간 피처 생성 및 ID 인코딩을 시작합니다...")
std = pd.read_parquet(OUTDIR / "combined.parquet") # 앞 단계 결과물 로드
std_time = add_time_features(std)

# --- [수정] Parquet 저장을 위한 임시 데이터프레임 생성 ---
# 원본 std_time의 타입을 바꾸지 않고, 저장을 위한 복사본을 만듭니다.
df_to_save = std_time.copy()

# datetime 타입은 종종 Parquet 변환 시 문제를 일으키므로,
# 가장 안전한 ISO 8601 표준 문자열 포맷으로 변환하여 저장합니다.
df_to_save["event_time"] = df_to_save["event_time"].astype(str)
# --------------------------------------------------------

save_table(std_time, OUTDIR / "combined_with_time.parquet")


# ===== vocab 생성 + ID 인코딩 =====
# 1) event_type 어휘집
vocab_event = build_vocab(std_time["event_type"], name="event_type", min_freq=1)
with open(OUTDIR / "event_type.vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab_event, f, ensure_ascii=False, indent=2)
print(f"[OK] 어휘집 저장: {OUTDIR / 'event_type.vocab.json'} (size={len(vocab_event['id2token'])})")

# 2) location_id 어휘집 (숫자지만 '코드'이므로 카테고리 취급)
vocab_loc = build_vocab(std_time["location_id"], name="location_id", min_freq=1)
with open(OUTDIR / "location_id.vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab_loc, f, ensure_ascii=False, indent=2)
print(f"[OK] 어휘집 저장: {OUTDIR / 'location_id.vocab.json'} (size={len(vocab_loc['id2token'])})")

# 3) ID 인코딩 컬럼 추가
encoded = std_time.copy()
encoded["event_type_id"]  = encode_with_vocab(encoded["event_type"],  vocab_event, "event_type")
encoded["location_id_id"] = encode_with_vocab(encoded["location_id"], vocab_loc,   "location_id")

# 저장
save_table(encoded, OUTDIR / "combined_encoded.parquet")


[OK] 저장(parquet): preprocessed/combined_with_time.parquet (shape=(920000, 22))


# 4. 모델 학습을 위한 준비 - 데이터 분할 및 스케일링

In [ ]:
# ===== 셀 4: 데이터 분할, 스케일링, 최종 저장 =====
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib # 스케일러 저장을 위해 사용

print("\n[INFO] 최종 피처 엔지니어링 및 데이터 분할/스케일링을 시작합니다...")

# 1. 셀 5에서 최종적으로 생성된 데이터를 불러옵니다.
print("[INFO] 스케일링을 위해 인코딩된 데이터를 불러옵니다.")
try:
    final_df = pd.read_parquet(OUTDIR / "combined_encoded.parquet")
except FileNotFoundError:
    # Parquet 저장 실패 시 생성된 CSV를 대신 읽음
    final_df = pd.read_csv(OUTDIR / "combined_encoded.csv")

# 2. 데이터를 학습(Train)과 테스트(Test) 세트로 분할합니다.
#    (시계열 데이터이므로 epc_code 단위로 분할하거나, 시간순으로 분할하는 것이 더 좋습니다.)
#    여기서는 간단한 예시로 랜덤 분할을 사용합니다. epc_code를 기준으로 분할해야 합니다.
epc_codes = final_df["epc_code"].unique()

# 2. 이 식별자 자체를 80:20으로 랜덤하게 나눕니다.
train_epcs, test_epcs = train_test_split(epc_codes, test_size=0.2, random_state=42)

train_df = final_df[final_df["epc_code"].isin(train_epcs)].copy()
test_df = final_df[final_df["epc_code"].isin(test_epcs)].copy()

print(f"[INFO] 데이터 분할 완료: Train={train_df.shape}, Test={test_df.shape}")

# 3. 스케일링 할 컬럼을 지정합니다.
cols_to_scale = ['delta_t_sec'] # 스케일 조정이 필요한 컬럼 목록

# 4. StandardScaler를 생성하고 '학습 데이터'에만 fit합니다.
scaler = StandardScaler()
print(f"[INFO] 학습 데이터의 '{cols_to_scale}' 컬럼에 스케일러를 학습(fit)합니다.")
# scaler.fit()은 2D 배열을 기대하므로, 대괄호를 두 번 사용합니다.
scaler.fit(train_df[cols_to_scale])

# 5. 학습된 스케일러로 Train, Test 데이터 모두를 transform합니다.
print("[INFO] 학습된 스케일러로 Train/Test 데이터를 변환(transform)합니다.")
train_df[cols_to_scale] = scaler.transform(train_df[cols_to_scale])
test_df[cols_to_scale] = scaler.transform(test_df[cols_to_scale])

# 6. 학습된 스케일러를 파일로 저장합니다. (어휘집 저장만큼 중요!)
scaler_path = OUTDIR / "delta_t_scaler.joblib"
joblib.dump(scaler, scaler_path)
print(f"[OK] 학습된 스케일러 저장 완료: {scaler_path}")

# 7. 스케일링까지 완료된 최종 데이터셋을 각각 저장합니다.
save_table(train_df, OUTDIR / "train_final.parquet")
save_table(test_df, OUTDIR / "test_final.parquet")

print("\n--- 스케일링 결과 확인 (Train Set) ---")
print(train_df[cols_to_scale].describe())

print("[OK] 전처리 최종 완료.")

[INFO] 스케일링을 위해 인코딩된 데이터를 불러옵니다.
[INFO] 데이터 분할 완료: Train=(736188, 24), Test=(183812, 24)
[INFO] 학습 데이터의 '['delta_t_sec']' 컬럼에 스케일러를 학습(fit)합니다.
[INFO] 학습된 스케일러로 Train/Test 데이터를 변환(transform)합니다.
[OK] 학습된 스케일러 저장 완료: preprocessed/delta_t_scaler.joblib
[OK] 저장(parquet): preprocessed/train_final.parquet (shape=(736188, 24))
[OK] 저장(parquet): preprocessed/test_final.parquet (shape=(183812, 24))

--- 스케일링 결과 확인 (Train Set) ---
        delta_t_sec
count  7.361880e+05
mean  -1.482493e-17
std    1.000001e+00
min   -5.533837e-01
25%   -5.508160e-01
50%   -5.353657e-01
75%    6.948806e-02
max    3.022292e+00
